In [2]:
!pip install pyaudio

Defaulting to user installation because normal site-packages is not writeable


In [3]:
pip install pydub

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip show pyaudio

Name: PyAudio
Version: 0.2.14
Summary: Cross-platform audio I/O with PortAudio
Home-page: https://people.csail.mit.edu/hubert/pyaudio/
Author: Hubert Pham
Author-email: 
License: MIT
Location: C:\Users\hp\AppData\Roaming\Python\Python311\site-packages
Requires: 
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [6]:
import pyaudio  # For recording audio input from the microphone

In [7]:
import os  # Provides functions to interact with the operating system (like file path

In [8]:
import time  # Used to track time, such as how long you've been recording

In [9]:
import threading  # Allows multiple processes to run simultaneously (like recording without freezing the UI)

In [10]:
import tkinter as tk  # For creating the graphical user interface 

In [11]:
# here we using pydub to convert the auio in .mp3 file 
from pydub import AudioSegment  # For audio manipulation and format conversion
from pydub.utils import make_chunks  # For breaking audio into smaller parts if needed
from io import BytesIO  # To handle in-memory audio data


In [12]:
class VoiceRecorder:
    def __init__(self):
        # Create the main window
        self.root = tk.Tk()
        self.root.title("Voice Recorder")  # Set the window title
        self.root.resizable(False, False)  # Prevent the window from being resized
        
        # Create a button with a microphone emoji 🎤, when clicked it will start/stop recording
        self.button = tk.Button(self.root, text="🎤", font=("Arial", 120, "bold"), command=self.click_handler)
        self.button.pack()  # Display the button on the window

        # Label to display the recording duration in "00:00:00" format
        self.label = tk.Label(self.root, text="00:00:00", font=("Arial", 24))
        self.label.pack()  # Display the label below the button
        
        self.recording = False  # Flag to check if we're currently recording or not
        self.frames = []  # Store audio frames in memory while recording
        self.root.mainloop()  # Start the Tkinter event loop, keeps the window open

    def click_handler(self):
        # Toggle recording state when the button is clicked
        if self.recording:
            self.recording = False  # Stop recording
            self.button.config(fg="black")  # Change the button text color to black when not recording
        else:
            self.recording = True  # Start recording
            self.button.config(fg="red")  # Change the button text color to red while recording
            # Run the record function in a separate thread to avoid freezing the interface
            threading.Thread(target=self.record).start()

    def record(self):
        audio = pyaudio.PyAudio()  # Create an audio object to interact with the audio device
        # Open an audio stream to record from the microphone
        stream = audio.open(format=pyaudio.paInt16, channels=1, rate=44100,
                            input=True, frames_per_buffer=1024)
        
        self.frames = []  # Clear previous frames
        start = time.time()  # Start time to track how long we've been recording

        while self.recording:
            data = stream.read(1024)  # Read 1024 samples from the microphone
            self.frames.append(data)  # Add the recorded data to the frames list

            # Calculate the time passed since recording started
            passed = time.time() - start
            secs = int(passed % 60)  # Calculate seconds
            mins = int((passed // 60) % 60)  # Calculate minutes
            hours = int(passed // 3600)  # Calculate hours
            # Update the label to display the current duration
            self.label.config(text=f"{hours:02d}:{mins:02d}:{secs:02d}")
        
        # Stop the audio stream when recording is finished
        stream.stop_stream()
        stream.close()  # Close the audio stream
        audio.terminate()  # Terminate the audio object (clean up)

        # Combine the frames into a single audio segment using pydub
        audio_segment = AudioSegment(
            data=b''.join(self.frames), 
            sample_width=2,  # pyaudio.paInt16 gives 2 bytes per sample
            frame_rate=44100,
            channels=1
        )

        # using this command to save the files in Downloads folder
        downloads_folder = os.path.join(os.path.expanduser("~"), "Downloads")
        
        # Create a unique filename
        exists = True
        i = 1
        while exists: 
            file_path_mp3 = os.path.join(downloads_folder, f"recording{i}.mp3")  # Create a new filename
            if os.path.exists(file_path_mp3):  # Check if the file already exists
                i += 1  # If it does, increment the number
            else:
                exists = False  # If it doesn't exist, stop the loop

        # Save the recorded data directly as an MP3 file
        audio_segment.export(file_path_mp3, format="mp3")  
        
        # we make this command to tell the user where the save files is sitiuated 
        print(f"Audio saved as {file_path_mp3}")

# to Run the Voice Recorder 
VoiceRecorder()

In [ ]:
#when you run this program so u c a icon appear in the icon bar if the mic does not show directly on the 
#screen just click that and then u c a mic icon or image on screen just the mic to on and off the recording 